In [1]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
import json 
# Lead chunks.json into a dictionary
with open('chunks.json', 'r') as f:
    chunks = json.load(f)

In [11]:
#Convert dict to a tuple of (game, review)
reviews = []
for game in chunks:
    for review in chunks[game]:
        reviews.append((game, review))


In [14]:
# Create a list for reviews and a list for games
review_list = []
game_list = []
for review in reviews:
    game_list.append(review[0])
    review_list.append(review[1])


['After 8 years playing it, I didn\'t improve my skills in-game.  However, I learned new language skills: now I can curse in Russian and Brazilian Portuguese.  Also, my tolerance level to guttural screams and loud sounds was increased.  Now I can bear the crying of my children for longer and I can make them feel better by singing sweet traditional songs from other cultures, such as   (Dark Eyes) or Garota de Ipanema (The Girl from Ipanema).  In conclusion, Counter Strike transformed me into a better father and a better person, in addition to making me more patient, tolerant to hateful people and more trained in languages and cultures.  Better than any psychiatric therapy. 10/10The time I spent playing this game could\'ve been spent... ...studying ...applying for jobs ...making friends irl ...getting a wife ...spending time with my family ...working out ...becoming a better man in general  From the time I spent in this game, I got... ...an extremely racist and homophobic vocabulary ...f

In [18]:
# Convert each game to a dictionary of gameDict["game"] = [game]
gameDictList = []
for game in game_list:
    gameDict = {}
    gameDict["game"] = [game]
    gameDictList.append(gameDict)
gameDictList

[{'game': ['Counter-Strike: Global Offensive']},
 {'game': ['Far Cry® 6']},
 {'game': ['Far Cry® 6']},
 {'game': ['Far Cry® 6']},
 {'game': ['Team Fortress 2']},
 {'game': ['Team Fortress 2']},
 {'game': ['Team Fortress 2']},
 {'game': ['Forza Horizon 5']},
 {'game': ['Forza Horizon 5']},
 {'game': ['Crusader Kings III']},
 {'game': ['Crusader Kings III']},
 {'game': ['Crusader Kings III']},
 {'game': ['Crusader Kings III']},
 {'game': ['Crusader Kings III']},
 {'game': ['Crusader Kings III']},
 {'game': ['Crusader Kings III']},
 {'game': ['Crusader Kings III']},
 {'game': ['Crusader Kings III']},
 {'game': ['EA SPORTS™ FIFA 23']},
 {'game': ['EA SPORTS™ FIFA 23']},
 {'game': ['EA SPORTS™ FIFA 23']},
 {'game': ['EA SPORTS™ FIFA 23']},
 {'game': ['EA SPORTS™ FIFA 23']},
 {'game': ['Hearts of Iron IV']},
 {'game': ['Hearts of Iron IV']},
 {'game': ['Hearts of Iron IV']},
 {'game': ['Hearts of Iron IV']},
 {'game': ['Hearts of Iron IV']},
 {'game': ['Hearts of Iron IV']},
 {'game': ['Hear

In [4]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [5]:
import os
import dotenv

dotenv.load_dotenv('env')
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_API_ENV = os.environ['PINECONE_API_ENV']

In [6]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [7]:
# initialize pinecone
pinecone.init(
    api_key=PINECONE_API_KEY,  # find at app.pinecone.io
    environment=PINECONE_API_ENV  # next to api key in console
)
index_name = "langchain" # put in the name of your pinecone index here

In [19]:
#docsearch = Pinecone.from_texts(review_list, embeddings, metadatas=gameDictList, index_name=index_name) #Initial population !!
docsearch = Pinecone.from_existing_index(index_name=index_name, embedding=embeddings)

In [20]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [21]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [33]:
query = """" I'm looking for a sandbox-style game with pixelated graphics that offers both single-player and multiplayer modes. I enjoy games where I can explore and modify an open world, digging into the earth, gathering resources, and building structures. I want a game with a progression system that's tied to combating various bosses and enemies. The game should include crafting mechanics, allowing me to utilize the resources I gather to create new items, tools, and weapons. I'd also appreciate if the game had different biomes and events to keep the gameplay engaging and dynamic."""
docs = docsearch.similarity_search_with_score(query, k=5)


In [34]:
docs[0]

(Document(page_content='I understand this is in early access and I hope things change and improve, but I read a bunch of reviews before purchasing and the game still wasn\'t what I expected. Am I missing something? I wanted to design and build my own log cabin, but the sandbox mode has me building a house in the suburbs for a "client" (and to that client\'s specifications), and the story mode (?) has me building a log cabin based on the game\'s blueprints. The sandbox should let me build whatever I want, wherever I want, imo. Maybe this will come later?   The above complaint is based on my proclivities specifically, but the game at its core also leaves something to be desired. I\'ve played the first three levels and there aren\'t any design choices I get to make. Let me do something besides click to advance a linear, pre-determined path.  Finally, this game\'s text could really use some proofreading. "Lenght" and "get rid of the bark of them" are just a couple errors I remember off the

In [36]:
doc = docs[0][0]  # The first item in the tuple is the Document object
metadata = doc.metadata  # This will be a dictionary with the metadata
game_value = metadata['game']  # Access the value of the 'game' key in the metadata

['House Builder']

In [ ]:
query = """" I'm looking for a sandbox-style game with pixelated graphics that offers both single-player and multiplayer modes. I enjoy games where I can explore and modify an open world, digging into the earth, gathering resources, and building structures. I want a game with a progression system that's tied to combating various bosses and enemies. The game should include crafting mechanics, allowing me to utilize the resources I gather to create new items, tools, and weapons. I'd also appreciate if the game had different biomes and events to keep the gameplay engaging and dynamic."""
docs = docsearch.similarity_search_with_score(query, k=5, filters={"game": [game_value]})

In [35]:
# get the page_content of the first n documents
first_docs = docs[0:1]
first_docs = [doc[0] for doc in first_docs]
first_docs


[Document(page_content='I understand this is in early access and I hope things change and improve, but I read a bunch of reviews before purchasing and the game still wasn\'t what I expected. Am I missing something? I wanted to design and build my own log cabin, but the sandbox mode has me building a house in the suburbs for a "client" (and to that client\'s specifications), and the story mode (?) has me building a log cabin based on the game\'s blueprints. The sandbox should let me build whatever I want, wherever I want, imo. Maybe this will come later?   The above complaint is based on my proclivities specifically, but the game at its core also leaves something to be desired. I\'ve played the first three levels and there aren\'t any design choices I get to make. Let me do something besides click to advance a linear, pre-determined path.  Finally, this game\'s text could really use some proofreading. "Lenght" and "get rid of the bark of them" are just a couple errors I remember off the

In [31]:
query = "Recommend this game to me, Start by telling the name of the game. Make relations to my query: " + query
chain.run(input_documents=first_docs, question=query)

" Unfortunately, based on the reviews and your proclivities, I would not recommend the game you mentioned. It does not seem to offer the features you are looking for. Instead, I would suggest you try Minecraft. It is a sandbox-style game with pixelated graphics that offers both single-player and multiplayer modes. You can explore and modify an open world, digging into the earth, gathering resources, and building structures. There is a progression system that's tied to combating various bosses and enemies. You can also craft items, tools, and weapons with the resources you gather. There are different biomes and events to keep the gameplay engaging and dynamic."